In [1]:
%load_ext autoreload
%autoreload 2

import os
from os.path import join, isfile, isdir
from urllib.request import urlretrieve
from anndata import read_h5ad
import scanpy as sc

from vitessce import (
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    #AnnDataWrapper
)

from vitessce.wrappers import AnnDataWrapper

from vitessce.data_utils import (
    optimize_adata,
    VAR_CHUNK_SIZE,
)

### some dataset preparation

In [2]:
adata_filepath = join("data", "habib17.processed.h5ad")
if not isfile(adata_filepath):
    os.makedirs("data", exist_ok=True)
    urlretrieve('https://covid19.cog.sanger.ac.uk/habib17.processed.h5ad', adata_filepath)
adata = read_h5ad(adata_filepath)
top_dispersion = adata.var["dispersions_norm"][
    sorted(
        range(len(adata.var["dispersions_norm"])),
        key=lambda k: adata.var["dispersions_norm"][k],
    )[-51:][0]
]
adata.var["top_highly_variable"] = (
    adata.var["dispersions_norm"] > top_dispersion
)

/Users/yufanxia/anaconda3/envs/vitessce-python-dev/lib/python3.9/site-packages/anndata/compat/__init__.py:311: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/Users/yufanxia/anaconda3/envs/vitessce-python-dev/lib/python3.9/site-packages/anndata/compat/__init__.py:311: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(
/var/folders/pz/x2v3y0f93k9d3nfx01z18ctr0000gn/T/ipykernel_4476/960494707.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  key=lambda k: adata.var["dispersions_norm"][k],
/var/folders/pz/x2v3y0f93k9d3nfx01z18ctr0000gn/T/ipykernel_4476/960494707.py:6: FutureWarning: Series.__getitem__ 

In [3]:
zarr_filepath = join("data", "habib17.processed.zarr")
if not isdir(zarr_filepath):
    adata = optimize_adata(
        adata,
        obs_cols=["CellType"],
        obsm_keys=["X_umap"],
        optimize_X=True,
        var_cols=["top_highly_variable"],
    )
adata.write_zarr(zarr_filepath, chunks=[adata.shape[0], VAR_CHUNK_SIZE])

### setup Anndatawrapper automatically and visualize

In [4]:
vc = VitessceConfig(schema_version="1.0.15", name='Habib et al', description='COVID-19 Healthy Donor Brain')
dataset = vc.add_dataset(name='Brain').add_object(AnnDataWrapper(
        adata_path=zarr_filepath,
        adata=adata,
    )
)

For mapping you can choose from:  ['X_umap']
For gene filtering the chosen filter is: var/top_highly_variable, specify yourself if this is not wanted


In [5]:
scatterplot = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="X_umap")
cell_sets = vc.add_view(cm.OBS_SETS, dataset=dataset)
genes = vc.add_view(cm.FEATURE_LIST, dataset=dataset)
heatmap = vc.add_view(cm.HEATMAP, dataset=dataset)
vc.layout((scatterplot | cell_sets) / (heatmap | genes));
vw = vc.widget()
vw

VitessceWidget(config={'version': '1.0.15', 'name': 'Habib et al', 'description': 'COVID-19 Healthy Donor Brai…